In [ ]:
#use the basic filtering methods first to remove quickly some unnnecessary features. then you can movie into other techniques to remove further features
#filter/remove columns based on your domain knowledge
#filter duplicate features
#filter one among correlated features 
#filter constant columns
#these are some basic methods u can use initiallu before going into ml based
#since all these are basic , discusses only variance threhold here in this notebook

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [4]:
df=pd.read_csv("train (1).csv")
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
#deleteing columns with null values just for easier demo of feature selection
s=df.isnull().sum()>0
c=s[s].index
df.drop(c,axis=1,inplace=True)

In [6]:
#lets select only these columns for faster computation
c=['OverallQual', 'YearBuilt', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF',
       'BsmtFullBath', 'FullBath', 'Fireplaces', 'GarageCars',
       'WoodDeckSF','MSZoning','Street','LotShape','OpenPorchSF']

In [7]:
x_train,x_test,y_train,y_test=train_test_split(df[c],df["SalePrice"],test_size=0.2,random_state=1)

In [8]:
trans=ColumnTransformer([("encoder",OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value=-1),x_train.select_dtypes(include="object").columns)],remainder="passthrough").set_output(transform="pandas")

In [9]:
x_train=trans.fit_transform(x_train)

In [10]:
x_test=trans.transform(x_test)

In [11]:
x_train.head()

,encoder__MSZoning,encoder__Street,encoder__LotShape,remainder__OverallQual,remainder__YearBuilt,remainder__BsmtUnfSF,remainder__1stFlrSF,remainder__2ndFlrSF,remainder__BsmtFullBath,remainder__FullBath,remainder__Fireplaces,remainder__GarageCars,remainder__WoodDeckSF,remainder__OpenPorchSF
921,3.0,1.0,3.0,5,1900,188,1272,928,2,2,0,0,0,70
520,3.0,1.0,3.0,4,1900,0,694,600,0,2,0,0,220,114
401,3.0,1.0,0.0,7,2005,1286,1310,0,0,2,1,2,0,0
280,3.0,1.0,3.0,7,1989,386,1175,807,0,2,1,2,0,84
1401,3.0,1.0,0.0,6,2004,80,864,729,1,2,1,2,100,75


# Wrapper methods

## 1. Forward Selection 

In [12]:
from sklearn.feature_selection import SequentialFeatureSelector

In [135]:
x_train1=x_train.copy()
x_test1=x_test.copy()

In [136]:
sfs=SequentialFeatureSelector(estimator=RandomForestRegressor(),direction="forward",scoring="r2",cv=2,n_features_to_select=5)

In [137]:
sfs.fit(x_train1,y_train)

SequentialFeatureSelector(cv=2, estimator=RandomForestRegressor(),
                          n_features_to_select=5, scoring='r2')

In [138]:
#these are the features 
sfs.get_feature_names_out()

array(['encoder__MSZoning', 'remainder__OverallQual',
       'remainder__1stFlrSF', 'remainder__GarageCars',
       'remainder__WoodDeckSF'], dtype=object)

In [139]:
#now lets try to select features by specifying tol, here we dont speacify how many features to select
x_train1=x_train.copy()
x_test1=x_test.copy()

In [ ]:
#tol:If the score is not incremented by at least `tol` between two 
#consecutive feature additions or removals, stop adding or removing. keep n_features_to_select="auto"

In [146]:
sfs=SequentialFeatureSelector(estimator=RandomForestRegressor(),direction="forward",tol=0.01,scoring="r2",cv=2,n_features_to_select="auto")

In [147]:
sfs.fit(x_train1,y_train)

SequentialFeatureSelector(cv=2, estimator=RandomForestRegressor(),
                          n_features_to_select='auto', scoring='r2', tol=0.01)

In [148]:
#these are the features 
sfs.get_feature_names_out()

array(['encoder__MSZoning', 'remainder__OverallQual',
       'remainder__GarageCars'], dtype=object)

## 2. Backward Elimination

In [39]:
x_train1=x_train.copy()
x_test1=x_test.copy()

In [44]:
#you can alson use tol 
sfs=SequentialFeatureSelector(estimator=RandomForestRegressor(),direction="backward",scoring="r2",cv=2,n_features_to_select=5).set_output(transform="pandas")

In [45]:
sfs.fit_transform(x_train1,y_train)

,remainder__OverallQual,remainder__YearBuilt,remainder__1stFlrSF,remainder__2ndFlrSF,remainder__BsmtFullBath
921,5.0,1900.0,1272.0,928.0,2.0
520,4.0,1900.0,694.0,600.0,0.0
401,7.0,2005.0,1310.0,0.0,0.0
280,7.0,1989.0,1175.0,807.0,0.0
1401,6.0,2004.0,864.0,729.0,1.0
...,...,...,...,...,...
715,6.0,1974.0,1350.0,0.0,0.0
905,5.0,1954.0,1063.0,0.0,1.0
1096,6.0,1914.0,773.0,582.0,0.0
235,6.0,1971.0,483.0,504.0,0.0


In [46]:
sfs.transform(x_test)

,remainder__OverallQual,remainder__YearBuilt,remainder__1stFlrSF,remainder__2ndFlrSF,remainder__BsmtFullBath
258,7.0,2001.0,963.0,829.0,0.0
267,5.0,1939.0,1052.0,720.0,0.0
288,5.0,1967.0,900.0,0.0,0.0
649,4.0,1970.0,630.0,0.0,1.0
1233,5.0,1959.0,1188.0,0.0,1.0
...,...,...,...,...,...
163,4.0,1956.0,882.0,0.0,0.0
47,8.0,2006.0,1656.0,0.0,0.0
1432,4.0,1927.0,968.0,0.0,0.0
98,5.0,1920.0,835.0,0.0,0.0


In [159]:
#these are the features 
sfs.get_feature_names_out()

array(['remainder__OverallQual', 'remainder__YearBuilt',
       'remainder__1stFlrSF', 'remainder__2ndFlrSF',
       'remainder__GarageCars'], dtype=object)

In [ ]:
#In forward feature selection, tol determines the minimum improvement in score required to continue the selection process.
#In backward feature selection, tol determines the maximum drop in score allowed during each iteration before the selection process stops.

## 3. Recusrsive Feature Elimination

In [181]:
x_train1=x_train.copy()
x_test1=x_test.copy()

In [161]:
from sklearn.feature_selection import RFE

In [162]:
rfe=RFE(estimator=RandomForestRegressor(),step=1,n_features_to_select=5)

In [163]:
rfe.fit(x_train1,y_train)

RFE(estimator=RandomForestRegressor(), n_features_to_select=5)

In [164]:
rfe.get_feature_names_out()

array(['remainder__OverallQual', 'remainder__BsmtUnfSF',
       'remainder__1stFlrSF', 'remainder__2ndFlrSF',
       'remainder__GarageCars'], dtype=object)

In [192]:
#recursive feature elimination with crossvalidation
from sklearn.feature_selection import RFECV

In [196]:
rfecv=RFECV(estimator=RandomForestRegressor(),scoring="r2",step=2,min_features_to_select=4,cv=2)

In [197]:
rfecv.fit(x_train1,y_train)

RFECV(cv=2, estimator=RandomForestRegressor(), min_features_to_select=4,
      scoring='r2', step=2)

In [198]:
rfecv.get_feature_names_out()

array(['encoder__MSZoning', 'remainder__OverallQual',
       'remainder__YearBuilt', 'remainder__BsmtUnfSF',
       'remainder__1stFlrSF', 'remainder__2ndFlrSF',
       'remainder__BsmtFullBath', 'remainder__FullBath',
       'remainder__Fireplaces', 'remainder__GarageCars',
       'remainder__WoodDeckSF', 'remainder__OpenPorchSF'], dtype=object)

#  Removing Constant/quasi constant Features using Variance Threshold

In [23]:
x_train1=x_train.copy()
x_test1=x_test.copy()

In [25]:
x_train1.shape

(1168, 14)

In [13]:
from sklearn.feature_selection import VarianceThreshold

In [36]:
#default threshold for variance is 0 means it is constant 
#but u can set threshold to remove low variance features/quasi constant features
#these constant/quasi constant features  doesnt convey any/much information
#here i am removing all the features which are having variance <1
vt=VarianceThreshold(threshold=1).set_output(transform="pandas")

In [37]:
vt.fit_transform(x_train1)

,encoder__LotShape,remainder__OverallQual,remainder__YearBuilt,remainder__BsmtUnfSF,remainder__1stFlrSF,remainder__2ndFlrSF,remainder__WoodDeckSF,remainder__OpenPorchSF
921,3.0,5.0,1900.0,188.0,1272.0,928.0,0.0,70.0
520,3.0,4.0,1900.0,0.0,694.0,600.0,220.0,114.0
401,0.0,7.0,2005.0,1286.0,1310.0,0.0,0.0,0.0
280,3.0,7.0,1989.0,386.0,1175.0,807.0,0.0,84.0
1401,0.0,6.0,2004.0,80.0,864.0,729.0,100.0,75.0
...,...,...,...,...,...,...,...,...
715,3.0,6.0,1974.0,1064.0,1350.0,0.0,0.0,0.0
905,3.0,5.0,1954.0,412.0,1063.0,0.0,0.0,0.0
1096,3.0,6.0,1914.0,684.0,773.0,582.0,136.0,0.0
235,3.0,6.0,1971.0,125.0,483.0,504.0,0.0,0.0


In [ ]:
#6 columns are removed which are having variance less than 1

In [35]:
#lets see the proof for this by checking variance of all columns
for i in x_train1.columns:
    print(i,":",x_train1[i].var())
    print("-"*20)

encoder__MSZoning : 0.37653552018406444
--------------------
encoder__Street : 0.002564091277247574
--------------------
encoder__LotShape : 2.015435902853581
--------------------
remainder__OverallQual : 1.8765179126903233
--------------------
remainder__YearBuilt : 917.5980678710235
--------------------
remainder__BsmtUnfSF : 201280.30671006936
--------------------
remainder__1stFlrSF : 156390.54111276436
--------------------
remainder__2ndFlrSF : 188592.2129971181
--------------------
remainder__BsmtFullBath : 0.27201156812339333
--------------------
remainder__FullBath : 0.29816529915131595
--------------------
remainder__Fireplaces : 0.42092474557171516
--------------------
remainder__GarageCars : 0.5396572114425277
--------------------
remainder__WoodDeckSF : 14923.550409520842
--------------------
remainder__OpenPorchSF : 4583.218872151955
--------------------


In [38]:
vt.transform(x_test)

,encoder__LotShape,remainder__OverallQual,remainder__YearBuilt,remainder__BsmtUnfSF,remainder__1stFlrSF,remainder__2ndFlrSF,remainder__WoodDeckSF,remainder__OpenPorchSF
258,3.0,7.0,2001.0,602.0,963.0,829.0,0.0,96.0
267,3.0,5.0,1939.0,342.0,1052.0,720.0,262.0,24.0
288,0.0,5.0,1967.0,432.0,900.0,0.0,0.0,0.0
649,3.0,4.0,1970.0,0.0,630.0,0.0,0.0,0.0
1233,0.0,5.0,1959.0,188.0,1188.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
163,3.0,4.0,1956.0,882.0,882.0,0.0,0.0,0.0
47,3.0,8.0,2006.0,1632.0,1656.0,0.0,0.0,146.0
1432,3.0,4.0,1927.0,656.0,968.0,0.0,0.0,0.0
98,3.0,5.0,1920.0,350.0,835.0,0.0,0.0,0.0


# Based on Feature Importance

In [16]:
#before finding feature importance it is important to scale the features in linear models

In [17]:
from sklearn.preprocessing import StandardScaler

In [20]:
scaler=StandardScaler().set_output(transform="pandas")

In [51]:
x_train1=x_train.copy()
x_test1=x_test.copy()

In [52]:
x_train1=scaler.fit_transform(x_train1)
x_test1=scaler.transform(x_test1)

In [44]:
x_train2=x_train1.copy()
x_test2=x_test1.copy()

###  Linear Models

In [1]:
from sklearn.feature_selection import SelectFromModel

In [50]:
from sklearn.linear_model import LinearRegression

In [55]:
sfm=SelectFromModel(LinearRegression()).set_output(transform="pandas")

In [56]:
sfm.fit_transform(x_train1,y_train)

,remainder__OverallQual,remainder__1stFlrSF,remainder__2ndFlrSF,remainder__GarageCars
921,-0.828481,0.242214,1.353929,-2.417038
520,-1.558795,-1.219992,0.598318,-2.417038
401,0.632147,0.338346,-0.783896,0.306648
280,0.632147,-0.003173,1.075182,0.306648
1401,-0.098167,-0.789931,0.895495,0.306648
...,...,...,...,...
715,-0.098167,0.439536,-0.783896,0.306648
905,-0.828481,-0.286507,-0.783896,-1.055195
1096,-0.098167,-1.020140,0.556852,-2.417038
235,-0.098167,-1.753773,0.377164,-1.055195


In [37]:
sfm.estimator_.coef_

array([-1715.9891723 ,   313.98552725, -3177.01917119, 30568.08763755,
        7016.97920151, -1662.03318376, 24810.73484102, 16306.18301917,
        5456.87544493,   238.12343523,  3995.2208242 , 10991.13527187,
        4739.36108884,  1491.38624437])

In [38]:
sfm.estimator_.feature_names_in_

array(['encoder__MSZoning', 'encoder__Street', 'encoder__LotShape',
       'remainder__OverallQual', 'remainder__YearBuilt',
       'remainder__BsmtUnfSF', 'remainder__1stFlrSF',
       'remainder__2ndFlrSF', 'remainder__BsmtFullBath',
       'remainder__FullBath', 'remainder__Fireplaces',
       'remainder__GarageCars', 'remainder__WoodDeckSF',
       'remainder__OpenPorchSF'], dtype=object)

In [39]:
sfm.estimator_.intercept_

182208.341609589

In [59]:
#now lets train the model and check its coefficients
model=LinearRegression().fit(x_train1,y_train)

In [60]:
model.coef_

array([-1715.9891723 ,   313.98552725, -3177.01917119, 30568.08763755,
        7016.97920151, -1662.03318376, 24810.73484102, 16306.18301917,
        5456.87544493,   238.12343523,  3995.2208242 , 10991.13527187,
        4739.36108884,  1491.38624437])

In [47]:
model.intercept_

182208.341609589

In [ ]:
#so we can see what the above select from model is doing is that it fits the estimator on training data and 
#it finds the coefficients and by defualt it selects features whose coefficient is greater than average. 

In [57]:
#we can also do by using setting a threshold on coefficient
sfm=SelectFromModel(LinearRegression(),threshold=20000).set_output(transform="pandas")
sfm.fit_transform(x_train1,y_train)

,remainder__OverallQual,remainder__1stFlrSF
921,-0.828481,0.242214
520,-1.558795,-1.219992
401,0.632147,0.338346
280,0.632147,-0.003173
1401,-0.098167,-0.789931
...,...,...
715,-0.098167,0.439536
905,-0.828481,-0.286507
1096,-0.098167,-1.020140
235,-0.098167,-1.753773


In [58]:
sfm.transform(x_test1)

,remainder__OverallQual,remainder__1stFlrSF
258,0.632147,-0.539484
267,-0.828481,-0.314335
288,-0.828481,-0.698859
649,-1.558795,-1.381897
1233,-0.828481,0.029714
...,...,...
163,-1.558795,-0.744395
47,1.362461,1.213646
1432,-1.558795,-0.526835
98,-0.828481,-0.863294


In [61]:
#same goes with the logistic regression during classification

In [62]:
#lets check with lasso regression which can make coefficients 0 also

In [63]:
from sklearn.linear_model import Lasso

In [78]:
#u can increase alpha further based on how many features required
sfm=SelectFromModel(Lasso(alpha=10000)).set_output(transform="pandas")
sfm.fit_transform(x_train1,y_train)

,remainder__OverallQual,remainder__1stFlrSF,remainder__2ndFlrSF,remainder__FullBath,remainder__Fireplaces,remainder__GarageCars,remainder__WoodDeckSF
921,-0.828481,0.242214,1.353929,0.784304,-0.955827,-2.417038,-0.775178
520,-1.558795,-1.219992,0.598318,0.784304,-0.955827,-2.417038,1.026481
401,0.632147,0.338346,-0.783896,0.784304,0.586170,0.306648,-0.775178
280,0.632147,-0.003173,1.075182,0.784304,0.586170,0.306648,-0.775178
1401,-0.098167,-0.789931,0.895495,0.784304,0.586170,0.306648,0.043758
...,...,...,...,...,...,...,...
715,-0.098167,0.439536,-0.783896,0.784304,0.586170,0.306648,-0.775178
905,-0.828481,-0.286507,-0.783896,-1.047831,-0.955827,-1.055195,-0.775178
1096,-0.098167,-1.020140,0.556852,-1.047831,-0.955827,-2.417038,0.338575
235,-0.098167,-1.753773,0.377164,-1.047831,-0.955827,-1.055195,-0.775178


In [79]:
sfm.estimator_.coef_

array([-0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  3.53041281e+04,
        0.00000000e+00, -0.00000000e+00,  1.38686555e+04,  1.83479923e+03,
        0.00000000e+00,  2.78447272e+03,  1.98678973e+03,  1.10930992e+04,
        3.01845793e+01,  0.00000000e+00])

In [80]:
#if u want to do with regularization in classification, logistic regression has parameter penalty="l1"

### Tree

In [81]:
#more the feature decreases impurity , the more important it is

In [82]:
#in random forest:high cardinal variable may show high importance since they are biased for this type of variable

In [83]:
from sklearn.ensemble import RandomForestRegressor

In [84]:
#by default selects features whose importance is greater than  mean of all feature importance
#can have threshold also
sfm=SelectFromModel(RandomForestRegressor()).set_output(transform="pandas")
sfm.fit_transform(x_train1,y_train)

,remainder__OverallQual,remainder__1stFlrSF
921,-0.828481,0.242214
520,-1.558795,-1.219992
401,0.632147,0.338346
280,0.632147,-0.003173
1401,-0.098167,-0.789931
...,...,...
715,-0.098167,0.439536
905,-0.828481,-0.286507
1096,-0.098167,-1.020140
235,-0.098167,-1.753773


In [88]:
sfm.estimator_.feature_importances_

array([7.88924197e-03, 1.14529158e-05, 3.97341541e-03, 5.82501153e-01,
       3.06477699e-02, 2.59162789e-02, 1.13742224e-01, 6.53092448e-02,
       5.16266739e-03, 3.34563217e-02, 1.58641759e-02, 6.86164404e-02,
       2.29525290e-02, 2.39570850e-02])

In [89]:
#only two features are True so that are selected
sfm.estimator_.feature_importances_>sfm.estimator_.feature_importances_.mean()

array([False, False, False,  True, False, False,  True, False, False,
       False, False, False, False, False])

# Statistical Methods

In [153]:
from sklearn.feature_selection import SelectKBest,chi2,f_classif,f_regression,mutual_info_classif,mutual_info_regression

In [140]:
#here it will be selected based on pvalue after chi2 test
c=SelectKBest(chi2,k=2).set_output(transform="pandas")

In [112]:
x_train1=x_train[["encoder__MSZoning","encoder__Street","encoder__LotShape","remainder__BsmtFullBath","remainder__FullBath"]]

In [130]:
x_train1=x_train1.astype("int")

In [128]:
y_train1=x_train['remainder__Fireplaces']

In [142]:
x_train1.head()

,encoder__MSZoning,encoder__Street,encoder__LotShape,remainder__BsmtFullBath,remainder__FullBath
921,3,1,3,2,2
520,3,1,3,0,2
401,3,1,0,0,2
280,3,1,3,0,2
1401,3,1,0,1,2


In [149]:
c.fit_transform(x_train1,y_train1)

,encoder__LotShape,remainder__BsmtFullBath
921,3,2
520,3,0
401,0,0
280,3,0
1401,0,1
...,...,...
715,3,0
905,3,1
1096,3,0
235,3,0


In [152]:
c.pvalues_

array([9.81831620e-01, 9.99783244e-01, 2.39966814e-12, 1.05940356e-04,
       4.49365011e-04])

In [145]:
c.feature_names_in_

array(['encoder__MSZoning', 'encoder__Street', 'encoder__LotShape',
       'remainder__BsmtFullBath', 'remainder__FullBath'], dtype=object)

In [139]:
c.scores_

array([1.72960524e-01, 8.74032768e-03, 5.71394192e+01, 2.09868479e+01,
       1.79549288e+01])

In [ ]:
#here this is the exmple of selection by chi square
#here both the target and feature should be categorical

In [ ]:
#use f_classif : anova test when feature is continuous and target is discrete (this is same as what we learnt)

In [ ]:
# use f_regression:  this is different method of anova (not what we learnt) , use this when target&feature is continuos  .
#this basically calculates correlation betweeen x and y, then converts it into F score and then to p-value

### Mutual Information

In [154]:
#here we determine mutual information between target and the feature and select the features which has high mutual information with target

In [155]:
#use mutual_info_classif for classification and mutual_info_regression for regression

In [157]:
s=SelectKBest(mutual_info_regression,k=2).set_output(transform="pandas")

In [158]:
s.fit_transform(x_train1,y_train)

,encoder__MSZoning,remainder__FullBath
921,3,2
520,3,2
401,3,2
280,3,2
1401,3,2
...,...,...
715,3,2
905,3,1
1096,4,1
235,4,1


In [159]:
#these are the mutual information values between each feature and target since i gave k=2 it selected only top2 features
mutual_info_regression(x_train1,y_train)

array([1.08022331e-01, 2.05807450e-04, 7.20728324e-02, 0.00000000e+00,
       2.69621948e-01])

In [162]:
s.feature_names_in_

array(['encoder__MSZoning', 'encoder__Street', 'encoder__LotShape',
       'remainder__BsmtFullBath', 'remainder__FullBath'], dtype=object)